# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
# choose a name for experiment
experiment_name = 'diabetes'

project_folder = './dibetes-project'

experiment = Experiment(ws, experiment_name)
experiment

quick-starts-ws-138275
aml-quickstarts-138275
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


Name,Workspace,Report Page,Docs Page
diabetes,quick-starts-ws-138275,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "capcpu"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                             max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded...................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "diabetes"
description_text = "diabetes prdection"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key]
         
df = dataset.to_pandas_dataframe()
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy =  BanditPolicy(slack_factor = 0.2, evaluation_interval=3, delay_evaluation=5)


#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    
"--C": choice(0.01,0.03,0.05,0.1) , 
"--max_iter": choice(range(100, 200)) })


#TODO: Create your estimator and hyperdrive config

if "training" not in os.listdir():
    os.mkdir("./training")

estimator = SKLearn(source_directory =
 './', entry_script='train.py', 
   compute_target='capcpu')

hyperdrive_run_config = HyperDriveConfig (estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='AUC_weighted',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=12,
                             max_concurrent_runs=4,
                             policy= early_termination_policy)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [10]:
#TODO: Submit your experiment

from azureml.widgets import RunDetails

hyperdrive_run = experiment.submit(hyperdrive_run_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_d5acea89-2c12-4b14-97ee-57b53565c235
Web View: https://ml.azure.com/experiments/diabetes/runs/HD_d5acea89-2c12-4b14-97ee-57b53565c235?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-138275/workspaces/quick-starts-ws-138275

Execution Summary
RunId: HD_d5acea89-2c12-4b14-97ee-57b53565c235
Web View: https://ml.azure.com/experiments/diabetes/runs/HD_d5acea89-2c12-4b14-97ee-57b53565c235?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-138275/workspaces/quick-starts-ws-138275

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "User errors were found in at least one of the child runs.",
    "messageFormat": null,
    "messageParameters": {},
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null
  },
  "correlation": null,
  "environment": null,
  "location": null,
  "time": "0001-01-01T0

{'runId': 'HD_d5acea89-2c12-4b14-97ee-57b53565c235',
 'target': 'capcpu',
 'status': 'Canceled',
 'startTimeUtc': '2021-02-09T20:04:43.456387Z',
 'endTimeUtc': '2021-02-09T20:17:10.574532Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "AUC_wei

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service